## Testing the irrigated extents MaxNDVI workflow

multithreaded version below 18/4/19

In [1]:
import numpy as np
import xarray as xr
import geopandas as gpd
import pandas as pd
from osgeo import gdal, ogr
import os
from multiprocessing import Pool
from rsgislib.segmentation import segutils
from rsgislib.segmentation import tiledsegsingle

import datacube 
from datacube.helpers import ga_pq_fuser
from datacube.storage import masking
from datacube.utils import geometry

#import custom functions
import sys
sys.path.append('src')
import DEAPlotting, SpatialTools
from transform_tuple import transform_tuple

In [2]:
############
#User Inputs
############

# where are the dcStats MaxNDVI tifs?
MaxNDVItiffs = "/g/data/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/New folder/maxndvi/"

# where are the dcStats NDVIArgMaxMin tifs?
NDVIArgMaxMintiffs = "/g/data/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/New folder/argmaxndvi/"

# where should I put the results?
results = '/g/data/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer/previous_run/testing_mosaics/results/'

#what season are we processing?
season = 'Summer'

#Input your area of interest's name
AOI = 'rsgislibe_test_tiled'

In [3]:
def irrigated_extent(tif):
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("starting processing of " + tif)
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    results_ = results
    if season == 'Summer':
        year = tif[9:13]
        nextyear = str(int(year) + 1)[2:] 
        year = year + "_" + nextyear
        year = season + year
        argmaxminyear = "NDVIArgMaxMin_" + year[6:10] + "1101.tif" 
    if season == 'Winter':
        year = tif[7:11]
        year = season + year
        argmaxminyear = "NDVIArgMaxMin_" + year[6:10] + "0501.tif" 

    #Creating a folder to keep things neat
    directory = results_ + AOI + "_" + year
    if not os.path.exists(directory):
        os.mkdir(directory)

    results_ = results_ + AOI + "_" + year + "/"
    
    #inputs to GDAL and RSGISlib
    InputNDVIStats = MaxNDVItiffs + tif
    KEAFile = results_ + AOI + '_' + year + '.kea'
    SegmentedKEAFile = results_ + AOI + '_' + year + '_sheperdSEG.kea'
    meanImage = results_ + AOI + '_' + year + "_ClumpMean.kea"
     
    # Change the tiff to a kea file
    gdal.Translate(KEAFile, InputNDVIStats, format='KEA', outputSRS='EPSG:3577')
    
    # Run segmentation, with creation of clump means
#     segutils.runShepherdSegmentation(KEAFile, SegmentedKEAFile,
#                         meanImage, numClusters=20, minPxls=100)

    tiledsegsingle.performTiledSegmentation(KEAFile, SegmentedKEAFile, tmpDIR='./tiledsegtmp'+year,
                                        tileWidth=1000, tileHeight=1000, minPxls=100)
    
    segment_means= xr.open_rasterio(meanImage).squeeze()
    
    #reclassify and threshold by different values
    a = np.where(segment_means.values>=0.8, 80, segment_means)
    b = np.where((a>=0.75) & (a<0.8), 75, a)
    c = np.where((b>=0.70) & (b<0.75), 70, b)
    d = np.where(c>=70, c, np.nan)
    
    print('exporting the multithreshold as Gtiff')
    transform, projection = transform_tuple(segment_means, (segment_means.x, segment_means.y), epsg=3577)
    #find the width and height of the xarray dataset we want to mask
    width,height = segment_means.shape
    
    SpatialTools.array_to_geotiff(results_ + AOI + "_" + year + "_multithreshold.tif",
                  d, geo_transform = transform, 
                  projection = projection, 
                  nodata_val=np.nan)
    
    #converting irrigated areas results to polygons
    print('converting multithreshold tiff to polygons...')
    multithresholdTIFF = results_ + AOI + "_" + year + "_multithreshold.tif"
    multithresholdPolygons = results_ + AOI + '_' + year + '_multithreshold.shp'
    
    os.system('gdal_polygonize.py ' + multithresholdTIFF + ' -f' + ' ' + '"ESRI Shapefile"' + ' ' + multithresholdPolygons)
    
    #filter by the area of the polygons to get rid of any forests etc
    print('filtering polygons by size, exporting, then rasterizing')
    gdf = gpd.read_file(multithresholdPolygons)
    gdf['area'] = gdf['geometry'].area
    smallArea = gdf['area'] <= 10000000
    gdf = gdf[smallArea]
    #export shapefile
    gdf.to_file(results_ + AOI + "_" + year + "_Irrigated.shp")
    
    gdf_raster = SpatialTools.rasterize_vector(results_ + AOI + "_" + year + "_Irrigated.shp",
                                               height, width, transform, projection, raster_path=None)
    
    print('loading, then masking timeof rasters')
    argmaxmin = xr.open_rasterio(NDVIArgMaxMintiffs+argmaxminyear)
    timeofmax = argmaxmin[0] 
    timeofmin = argmaxmin[1]

    # mask timeof layers by irrigated extent
    timeofmax = timeofmax.where(gdf_raster)
    timeofmin = timeofmin.where(gdf_raster)

    # export masked timeof layers.
    print('exporting the timeofmaxmin Gtiffs')
    SpatialTools.array_to_geotiff(results_ + AOI + "_" + year + "_timeofmaxNDVI.tif",
                  timeofmax.values,
                  geo_transform = transform, 
                  projection = projection, 
                  nodata_val=-9999)

    SpatialTools.array_to_geotiff(results_ + AOI + "_" + year + "_timeofminNDVI.tif",
                  timeofmin.values,
                  geo_transform = transform, 
                  projection = projection, 
                  nodata_val=-9999)
    

In [ ]:
maxNDVItiffFiles = os.listdir(MaxNDVItiffs)

if __name__ == '__main__':
    pool = Pool(os.cpu_count()-5)  #use 3 cpus on vdi
    pool.map(irrigated_extent, maxNDVItiffFiles)

In [4]:
maxNDVItiffFiles = os.listdir(MaxNDVItiffs)

irrigated_extent(maxNDVItiffFiles[0])



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
starting processing of ndvi_max_20151101.tif
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
./tiledsegtmpSummer2015_16/s1tilessegs_154f30/rsgislibe_test_tiled_Summer2015_16_154f30_S1TileC8_segs.kea
Stretch Input Image
Add 1 to stretched file to ensure there are no all zeros (i.e., no data) regions created.
Create Input Image Mask.
Mask stretched Image.
Deleting file: ./tiledsegtmpSummer2015_16/S1TileC8_segstemp/rsgislibe_test_tiled_Summer2015_16_154f30_S1TileC8_stchdonly.kea
Deleting file: ./tiledsegtmpSummer2015_16/S1TileC8_segstemp/rsgislibe_test_tiled_Summer2015_16_154f30_S1TileC8_stchdonlyOff.kea
Deleting file: ./tiledsegtmpSummer2015_16/S1TileC8_segstemp/rsgislibe_test_tiled_Summer2015_16_154f30_S1TileC8_stchdmaskonly.kea
Performing KMeans.
Apply KMeans to image.
Eliminate Single Pixels.
Perform clump.
Eliminate small pixels.
Relabel clumps.
Calculate image statistics and build pyramids.
Deleting file: ./tiledsegtmpSummer2015_16/S1TileC8_segstemp/rsgislibe_test_tiled_

OSError: [Errno 16] Device or resource busy: '.nfs2a800489b2013cd2000002ea'

In [ ]:
data = xr.open_rasterio('path/to/data').squeeze()
area = np.count_nonzero(~np.isnan(data.values))*(25*25) / 10000
area